In [1]:
import random
import re
import time
from typing import List, Tuple

import kscope
import pandas as pd
from metrics import map_ag_news_int_labels, report_metrics
from transformers import AutoTokenizer
from utils import get_label_token_ids, get_label_with_highest_likelihood, split_prompts_into_batches

# Getting Started

There is a bit of documentation on how to interact with the large models [here](https://kaleidoscope-sdk.readthedocs.io/en/latest/). The relevant github links to the SDK are [here](https://github.com/VectorInstitute/kaleidoscope-sdk) and underlying code [here](https://github.com/VectorInstitute/kaleidoscope).

First we connect to the service through which we'll interact with the LLMs and see which models are avaiable to us

In [2]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=6001)

Show all supported models

In [3]:
client.models

['OPT-175B', 'OPT-6.7B']

Show all model instances that are currently active

In [4]:
client.model_instances

[{'id': 'f6e55e38-2c01-4def-aaf9-d4531f9a2598',
  'name': 'OPT-175B',
  'state': 'ACTIVE'},
 {'id': '4213e687-fdaf-4046-a03c-ca59b484ad14',
  'name': 'OPT-6.7B',
  'state': 'ACTIVE'}]

To start, we obtain a handle to a model. In this example, let's use the OPT-175B model.

In [5]:
model = client.load_model("OPT-175B")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. So we set a number of important parameters. For a discussion of the configuration parameters see: `src/reference_implementations/prompting_vector_llms/CONFIG_README.md`

In [6]:
short_generation_config = {"max_tokens": 2, "top_k": 4, "top_p": 1.0, "rep_penalty": 1.0, "temperature": 1.0}

Let's try a basic prompt for factual information.

__Note__ that if you run the cell multiple times, you'll get different responses due to sampling.

In [7]:
generation = model.generate("What is the capital of Canada?", short_generation_config)
# Extract the text from the returned generation
generation.generation["text"]

['\nMont']

We're going to have our model attempt to classify some news articles from the AG News Dataset. Articles have a single label 1-4

1. World
2. Sports
3. Business
4. Sci/Tech

This is a constrained label space. We'll use the words "World", "Sports", "Business", and "Technology" as generative LM targets for each of the labels.

In [8]:
def remove_markup(text: str) -> str:
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    return text


def ag_news_processor(path: str) -> Tuple[List[str], List[str], List[str]]:
    ag_news_data = pd.read_csv(path)
    labels = ag_news_data["Class Index"].tolist()
    titles = ag_news_data["Title"].apply(lambda x: remove_markup(x)).tolist()
    descriptions = ag_news_data["Description"].apply(lambda x: remove_markup(x)).tolist()
    return labels, titles, descriptions


int_to_label_map = {1: "world", 2: "sports", 3: "business", 4: "technology"}
ag_news_labels, ag_news_titles, ag_news_descriptions = ag_news_processor(
    "resources/ag_news_datasets/ag_news_sample.csv"
)

In [9]:
ag_news_labels = map_ag_news_int_labels(ag_news_labels, int_to_label_map)
ag_news_descriptions = [description.replace("\\", " ").strip() for description in ag_news_descriptions]
ag_news_titles = [title.strip() for title in ag_news_titles]
label_words = ["World", "Sports", "Business", "Technology"]

In [10]:
model_input_texts = [
    f"Title: {ag_news_title} Description: {ag_news_description}"
    for ag_news_title, ag_news_description in zip(ag_news_titles, ag_news_descriptions)
]

Let's start by trying out a basic instruction prompt to see what the model does. You might also try some prompts from [this paper](https://arxiv.org/pdf/2212.04037.pdf). See Table 1.

In [11]:
prompt_template = "To which category does this news article belong?"
sample_texts = [f"{model_input_text} {prompt_template}" for model_input_text in model_input_texts[0:3]]
generation = model.generate(sample_texts, short_generation_config)
for text in generation.generation["text"]:
    print(text)
    print("==================================")

 (Check
 Desktop News





Not well...Now let's try to constrain the model a bit by including the desired labels in the instruction.

In [12]:
prompt_template = "From World, Sports, Business, Technology, the category is "
sample_texts = [f"{model_input_text} {prompt_template}" for model_input_text in model_input_texts[0:3]]
generation = model.generate(sample_texts, short_generation_config)
for text in generation.generation["text"]:
    print(text)
    print("==================================")

 Next

 Law,
____?


The model doesn't really answer in the space that we want it to. Let's try with some few-shot examples to see if that helps.

__NOTE__: We have simply randomly picked the examples used in the 5-shot prompt. Different choices might be made, including 4-shot or 8-shot prompts so that categories are evenly represented.

In [13]:
prompt_template_prefix = """Title: Lane drives in winning run in ninth Description: Jason Lane took an unusual post-game batting practice with hitting coach Gary Gaetti after a disappointing performance Friday night. Category (World, Sports, Business, Technology): Sports
Title: Arson attack on Jewish centre in Paris (AFP) Description: AFP - A Jewish social centre in central Paris was destroyed by fire overnight in an anti-Semitic arson attack, city authorities said. Category (World, Sports, Business, Technology): World
Title: Oil prices look set to dominate Description: The price of oil looks set to grab headlines as analysts forecast that its record-breaking run may well continue. Category (World, Sports, Business, Technology): Business
Title: Indexes in Japan fall short of hype Description: Japanese stocks have failed to measure up to an assessment made in April by Merrill Lynch #39;s chief global strategist, David Bowers, who said Japan was  quot;very much everyone #39;s favorite equity market. Category (World, Sports, Business, Technology): Business
Title: UK Scientists Allowed to Clone Human Embryos (Reuters) Description: Reuters - British scientists said on Wednesday they had received permission to clone human embryos for medical research, in what they believe to be the first such license to be granted in Europe. Category (World, Sports, Business, Technology): Technology
"""  # noqa
prompt_template_postfix = "Category (World, Sports, Business, Technology):"
sample_texts = [
    f"{prompt_template_prefix}{model_input_text} {prompt_template_postfix}"
    for model_input_text in model_input_texts[0:3]
]
generation = model.generate(sample_texts, short_generation_config)
for text in generation.generation["text"]:
    print(text)
    print("==================================")

 Business

 Business

 Sports


Few-shot learning definitely helps a lot! We'll measure accuracy on a sample of the AG news dataset below. However, there is nothing stoping the model from not selecting our labels. So can we do better? We can work around this by understanding the likelihood of our labels from the models perspective.

In [14]:
# We're interested in the activations from the last layer of the model, because this will allow us to calculate the
# likelihoods
last_layer_name = model.module_names[-1]
last_layer_name

'decoder.output_projection'

The last layer of the model corresponds to the probabilities of each token in the model vocabulary. That is, it is the conditional probability
$$
P(y_t \vert y_{<t}, x),
$$
The probability distribution over the vocabulary of the next token given the preceding tokens $y_{<t}$, and the prompt text $x$. Thus, for each token $y_{t}$ in our input, we get back a 50K vector corresponding to the probabilities over the vocabulary of $y_{t+1}$. We only care about the last token in our input, as it houses the probability of the, as yet, unseen token the model will generate.


Need to instantiate a tokenizer to obtain appropriate token indices for our labels. 

__NOTE__: All OPT models, regardless of size, used the same tokenizer. However, if you want to use a different type of model, a different tokenizer may be needed.

In [15]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
label_token_ids = get_label_token_ids(tokenizer, prompt_template, label_words)
# If you ever need to move back from token ids, you can use tokenizer.decode or tokenizer.batch_decode
tokenizer.decode(label_token_ids)

' World Sports Business Technology'

We need the token ids of our labels to exact the probabilties from the vocabulary of the model. The token id corresponds to the index of the token in the vocabulary matrix of the underlying model.

Let's look at how we can extract the likelihoods given the label tokens

In [16]:
single_prompted_input = f"{model_input_texts[0]} {prompt_template}"
# Create a prompt with one of the label words as a completion
activations = model.get_activations(single_prompted_input, [last_layer_name], short_generation_config)

In [17]:
last_layer_matrix = activations.activations[0][last_layer_name]
# The shape of this tensor should be number of input tokens by the vocabulary size (n x 50272)
print(f"Activations matrix shape: {last_layer_matrix.shape}")
predicted_label = get_label_with_highest_likelihood(
    last_layer_matrix, label_token_ids, int_to_label_map, right_shift=True
)
print(f"Predicted Label: {predicted_label}")

Activations matrix shape: torch.Size([61, 50272])
Predicted Label: business


## Accuracy

Time to compare our results across three methods. 
1. Measure the accuracy of our few-shot prompting approach.
2. Measure the accuracy of our likelihood approach without few-shot.
3. Measure the accuracy of our likelihood approach with few-shot.

In [18]:
lowercase_labels = [word.lower() for word in label_words]

### Few-shot only

In this example, we'll use a 5-shot prompt, as we did above and perform a "exact match" with our label space. That is, we parse out the first token that the model produces in its generation and simply try to string match it to one of our four label strings.

__Note__: Our generation configuration uses a `top_k = 4` which means that it picks from the top 4 vocabulary words (scaled by their relative probabilities). You could, try changing this to 1 to get a bit better factual extraction.

In [19]:
prompt_template_prefix = """Title: Lane drives in winning run in ninth Description: Jason Lane took an unusual post-game batting practice with hitting coach Gary Gaetti after a disappointing performance Friday night. Category (World, Sports, Business, Technology): Sports
Title: Arson attack on Jewish centre in Paris (AFP) Description: AFP - A Jewish social centre in central Paris was destroyed by fire overnight in an anti-Semitic arson attack, city authorities said. Category (World, Sports, Business, Technology): World
Title: Oil prices look set to dominate Description: The price of oil looks set to grab headlines as analysts forecast that its record-breaking run may well continue. Category (World, Sports, Business, Technology): Business
Title: Indexes in Japan fall short of hype Description: Japanese stocks have failed to measure up to an assessment made in April by Merrill Lynch #39;s chief global strategist, David Bowers, who said Japan was  quot;very much everyone #39;s favorite equity market. Category (World, Sports, Business, Technology): Business
Title: UK Scientists Allowed to Clone Human Embryos (Reuters) Description: Reuters - British scientists said on Wednesday they had received permission to clone human embryos for medical research, in what they believe to be the first such license to be granted in Europe. Category (World, Sports, Business, Technology): Technology
"""  # noqa
prompt_template_postfix = "Category (World, Sports, Business, Technology):"
prompts = [
    f"{prompt_template_prefix}{model_input_text} {prompt_template_postfix}" for model_input_text in model_input_texts
]
prompt_batches = split_prompts_into_batches(prompts)
predicted_labels = []
for batch_num, prompt_batch in enumerate(prompt_batches):
    generation = model.generate(prompt_batch, short_generation_config)
    print(f"Batch number {batch_num+1} Complete")
    # We'll use tokens this time and consider just the first token
    first_predicted_tokens = [tokens[0].strip().lower() for tokens in generation.generation["tokens"]]
    # If a token doesn't correspond to one of our labels, we'll randomly select one and count how many times that
    # happens for reporting
    n_no_match = 0
    for potential_prediction in first_predicted_tokens:
        if potential_prediction in lowercase_labels:
            predicted_labels.append(potential_prediction)
        else:
            n_no_match += 1
            print(f"Potential Prediction: {potential_prediction} does not match any label")
            predicted_labels.append(random.choice(lowercase_labels))

Batch number 1 Complete
Batch number 2 Complete
Batch number 3 Complete
Potential Prediction: science does not match any label
Batch number 4 Complete
Batch number 5 Complete
Batch number 6 Complete
Batch number 7 Complete
Batch number 8 Complete
Potential Prediction: politics does not match any label
Potential Prediction: politics does not match any label
Batch number 9 Complete
Potential Prediction: politics does not match any label
Batch number 10 Complete


In [20]:
report_metrics(predicted_labels, ag_news_labels, labels_order=["world", "sports", "business", "technology"])

Prediction Accuracy: 0.67
Confusion Matrix with ordering ['world', 'sports', 'business', 'technology']
[[14  3  8  3]
 [ 4 15  2  0]
 [ 2  0 19  2]
 [ 2  0  7 19]]
Label: world, F1: 0.56, Precision: 0.5, Recall: 0.6363636363636364
Label: sports, F1: 0.7692307692307692, Precision: 0.7142857142857143, Recall: 0.8333333333333334
Label: business, F1: 0.6440677966101694, Precision: 0.8260869565217391, Recall: 0.5277777777777778
Label: technology, F1: 0.7307692307692307, Precision: 0.6785714285714286, Recall: 0.7916666666666666


### Likelihood No Few-shot

In this example, we do not incorporate any demonstrations into the prompt (zero-shot prompt). From our experience above, the model does not do a good job generating responses that correspond to our label space. So rather than trying to match responses to our labels as strings, we extract the probabilties of our labels (see example above), as estimated by the model's verbalizer, and select the label with the highest probability as the prediction.

In [21]:
prompts = [f"{model_input_text} {prompt_template}" for model_input_text in model_input_texts]
# For memory management, we split the prompts into batches of size 10
predicted_labels = []
prompt_batches = split_prompts_into_batches(prompts)
for batch_num, prompt_batch in enumerate(prompt_batches):
    activations = model.get_activations(prompt_batch, [last_layer_name], short_generation_config)
    print(f"Batch number {batch_num+1} Complete")
    for activations_single_prompt in activations.activations:
        last_layer_matrix = activations_single_prompt[last_layer_name]
        predicted_label = get_label_with_highest_likelihood(
            last_layer_matrix, label_token_ids, int_to_label_map, right_shift=True
        )
        predicted_labels.append(predicted_label)

Batch number 1 Complete
Batch number 2 Complete
Batch number 3 Complete
Batch number 4 Complete
Batch number 5 Complete
Batch number 6 Complete
Batch number 7 Complete
Batch number 8 Complete
Batch number 9 Complete
Batch number 10 Complete


In [22]:
report_metrics(predicted_labels, ag_news_labels, labels_order=["world", "sports", "business", "technology"])

Prediction Accuracy: 0.43
Confusion Matrix with ordering ['world', 'sports', 'business', 'technology']
[[26  1  1  0]
 [17  4  0  0]
 [13  0 10  0]
 [20  0  5  3]]
Label: world, F1: 0.5, Precision: 0.9285714285714286, Recall: 0.34210526315789475
Label: sports, F1: 0.3076923076923077, Precision: 0.19047619047619047, Recall: 0.8
Label: business, F1: 0.5128205128205128, Precision: 0.43478260869565216, Recall: 0.625
Label: technology, F1: 0.19354838709677416, Precision: 0.10714285714285714, Recall: 1.0


### Likelihood with Few-Shot

The zero-shot prompt combined with likelihood estimation for our label space doesn't do a great job, but it is likely a lot better than if we had tried to exact match the generation. Let's combine the two approaches. We'll use a 5-shot prompt, as we did in the exact match example above, but now we'll use likelihood over our labels as the prediction mechanism rather than exact matching the first generated token.

In [23]:
prompt_template_prefix = """Title: Lane drives in winning run in ninth Description: Jason Lane took an unusual post-game batting practice with hitting coach Gary Gaetti after a disappointing performance Friday night. Category (World, Sports, Business, Technology): Sports
Title: Arson attack on Jewish centre in Paris (AFP) Description: AFP - A Jewish social centre in central Paris was destroyed by fire overnight in an anti-Semitic arson attack, city authorities said. Category (World, Sports, Business, Technology): World
Title: Oil prices look set to dominate Description: The price of oil looks set to grab headlines as analysts forecast that its record-breaking run may well continue. Category (World, Sports, Business, Technology): Business
Title: Indexes in Japan fall short of hype Description: Japanese stocks have failed to measure up to an assessment made in April by Merrill Lynch #39;s chief global strategist, David Bowers, who said Japan was  quot;very much everyone #39;s favorite equity market. Category (World, Sports, Business, Technology): Business
Title: UK Scientists Allowed to Clone Human Embryos (Reuters) Description: Reuters - British scientists said on Wednesday they had received permission to clone human embryos for medical research, in what they believe to be the first such license to be granted in Europe. Category (World, Sports, Business, Technology): Technology
"""  # noqa
prompt_template_postfix = "Category (World, Sports, Business, Technology):"
prompts = [
    f"{prompt_template_prefix}{model_input_text} {prompt_template_postfix}" for model_input_text in model_input_texts
]
# For memory management, we split the prompts into batches of size 10
predicted_labels = []
prompt_batches = split_prompts_into_batches(prompts)
for batch_num, prompt_batch in enumerate(prompt_batches):
    activations = model.get_activations(prompt_batch, [last_layer_name], short_generation_config)
    print(f"Batch number {batch_num+1} Complete")
    for activations_single_prompt in activations.activations:
        last_layer_matrix = activations_single_prompt[last_layer_name]
        predicted_label = get_label_with_highest_likelihood(
            last_layer_matrix, label_token_ids, int_to_label_map, right_shift=True
        )
        predicted_labels.append(predicted_label)

Batch number 1 Complete
Batch number 2 Complete
Batch number 3 Complete
Batch number 4 Complete
Batch number 5 Complete
Batch number 6 Complete
Batch number 7 Complete
Batch number 8 Complete
Batch number 9 Complete
Batch number 10 Complete


In [24]:
report_metrics(predicted_labels, ag_news_labels, labels_order=["world", "sports", "business", "technology"])

Prediction Accuracy: 0.74
Confusion Matrix with ordering ['world', 'sports', 'business', 'technology']
[[17  1  8  2]
 [ 0 21  0  0]
 [ 1  0 22  0]
 [ 0  0 14 14]]
Label: world, F1: 0.7391304347826088, Precision: 0.6071428571428571, Recall: 0.9444444444444444
Label: sports, F1: 0.9767441860465117, Precision: 1.0, Recall: 0.9545454545454546
Label: business, F1: 0.6567164179104478, Precision: 0.9565217391304348, Recall: 0.5
Label: technology, F1: 0.6363636363636364, Precision: 0.5, Recall: 0.875
